## VGG

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Loading VGG16 model

base_model = VGG16(weights="imagenet", include_top=False, input_tensor=input2)
base_model.trainable = False

vgg_model = Sequential()

for x in base_model.layers[:4]:
    vgg_model.add(x)

vgg_last = vgg_model.layers[-1].output
conv1 = Conv2D(100, (4, 4), activation='relu')(vgg_last)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(64, (2, 2), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)

vgg_output_reshaped = tf.keras.layers.Reshape((-1, 64), name='layer_taken_out_for_attentions')(pool2)

sample_model = Model(inputs = vgg_model.layers[0].output, outputs=vgg_output_reshaped)

sample_model.summary()

## Building cross attention layer model

In [ ]:

cross_attention_layer = MultiHeadAttention(key_dim=64, num_heads=8)(vgg_output_reshaped, lstm_layer2)

pool = layers.GlobalAveragePooling1D()(cross_attention_layer)

out1 = layers.LayerNormalization(epsilon=1e-6)(pool)

ffn_output = Sequential(
            [layers.Dense(64, activation="relu"), layers.Dense(32)]
        )(out1)

out2 = layers.LayerNormalization(epsilon=1e-6)(ffn_output)

final = Dense(1, activation='sigmoid')(out2)

cross_model = Model(inputs = [input1, input2], outputs = final)
cross_model.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="cross_attention_model_vgg.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

cross_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

cross_model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor, checkpointer],
            validation_data=([text_validate, image_data_validate], y_validate))

In [ ]:
cross_model.load_weights("cross_attention_model_vgg.hdf5")
pred = cross_model.predict([text_test, image_data_test])

pred = (pred > 0.5)

### Results

In [ ]:
print(classification_report(y_test, pred))